In [1]:
python3.10 -m venv tf_env


SyntaxError: invalid syntax (2520636081.py, line 1)

In [5]:
!pip install datasets transformers sentence-transformers nltk accelerate -U tensorflow==2.12.0

  Using cached datasets-3.0.2-py3-none-any.whl.metadata (20 kB)
  Using cached transformers-4.46.1-py3-none-any.whl.metadata (44 kB)
  Using cached sentence_transformers-3.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached accelerate-1.0.1-py3-none-any.whl.metadata (19 kB)
ERROR: Could not find a version that satisfies the requirement tensorflow==2.12.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.12.0

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 381.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 5.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [29]:
import numpy as np
from transformers import EarlyStoppingCallback, Trainer
import json, os
import torch
import torch.nn.functional as F
import re
import pandas as pd
import logging

from sklearn.model_selection import train_test_split
from datasets import load_dataset
from datasets import Dataset
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')


# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
# Load dataset
data = load_dataset("goendalf666/sales-conversations")

questions = []
answers = []
contexts = []

for row in data['train']:
    instruction = row.get("instruction", "You are the salesman. Answer the customer queries accordingly.")
    for i in range(0, 20, 2):
        customer = row.get(str(i), None)
        salesman = row.get(str(i + 1), None)

        if customer and salesman:
            questions.append(customer)
            answers.append(salesman)
            contexts.append(instruction)


qa_data = pd.DataFrame({
    'instruction': contexts,
    'question': questions,
    'answer': answers
})

qa_data.head()

,instruction,question,answer
0,You are the salesman. Answer the customer quer...,"Customer: Hi, Im interested in purchasing a ne...","Salesman: Of course, Id be happy to assist you..."
1,You are the salesman. Answer the customer quer...,Customer: Im looking for a laptop that can han...,Salesman: Absolutely! In order to find the bes...
2,You are the salesman. Answer the customer quer...,Customer: Im considering upgrading my companys...,Salesman: Thank you for considering us for you...
3,You are the salesman. Answer the customer quer...,Customer: Im in the market for a new smartwatc...,"Salesman: Absolutely! Before we proceed, could..."
4,You are the salesman. Answer the customer quer...,Customer: Im looking to buy a new printer for ...,Salesman: Certainly! To assist you in finding ...


In [4]:
# split data
qa_data = qa_data.head(500)
train_df, remaining = train_test_split(qa_data, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(remaining, test_size=0.5, random_state=42)

print("Training set size:", len(train_df))
print("Validation set size:", len(val_df))
print("Test set size:", len(test_df))

Training set size: 350
Validation set size: 75
Test set size: 75


In [5]:
train_df['answer']

5      Salesman: Great! Before I can recommend the id...
116    Salesman: Absolutely! We have a range of suppl...
45     Salesman: Great! Im glad you reached out. My n...
16     Salesman: Great question! We differentiate our...
462    Salesman: Of course! Our health product is des...
                             ...                        
106    Salesman: Absolutely! We have a range of diges...
270    Salesman: Its natural to have concerns about m...
348    Salesman: Absolutely, understanding your indus...
435    Salesman: Certainly! We have a stress manageme...
102    Salesman: Absolutely! We specialize in providi...
Name: answer, Length: 350, dtype: object

In [6]:
# map labels to integers
all_answers = pd.concat([train_df['answer'], val_df['answer'], test_df['answer']]).tolist()
all_answers = [x for x in all_answers if pd.notna(x)]
unique_labels = list(dict.fromkeys(all_answers))
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}

with open("label_mapping.json", "w") as f:
    json.dump(label_mapping, f)

In [7]:
# save label mapping to file for inference later
sent_transf = SentenceTransformer('all-MiniLM-L6-v2', device=device)
response = sent_transf.encode([key for key in label_mapping.keys() if key is not np.nan])
response_path = "./response_embeddings.npy"
np.save(response_path, response)
sent_transf_path = "./sent_transf"
os.makedirs(sent_transf_path, exist_ok=True)
sent_transf.save(sent_transf_path)


In [30]:
# set up semantic search
def semantic_search(query, top_k=5):
    query_embedding = sent_transf.encode([query])
    similarities = cosine_similarity(query_embedding, response)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    valid_keys = [key for key in label_mapping.keys() if key is not np.nan]

    return [(valid_keys[i], similarities[i]) for i in top_indices]

In [31]:
# set up normalization
def normalize_text(text):
    text = str(text).lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [33]:
# map labels
def map_label(example):
    if 'answer' in example:
        normalized_answer = normalize_text(example['answer'])
        normalized_label_mapping = {normalize_text(k): v for k, v in label_mapping.items()}
        example['label'] = normalized_label_mapping.get(normalized_answer, -1)
        # print(f"Assigned label: {example['label']}, Original answer: {example['answer']}")
    else:
        print("Warning: 'answer' column not found in the DataFrame.")
        example['label'] = -1
    return example

In [34]:
# Normalize keys
normalized_label_mapping = {normalize_text(k): v for k, v in label_mapping.items()}
train_df = train_df.apply(map_label, axis=1)
val_df = val_df.apply(map_label, axis=1)
test_df = test_df.apply(map_label, axis=1)


print("\nSample labels from train_df:")
for i in range(10):
    print(f"Example {i}: Label - {train_df.iloc[i]['label']}, Answer - {train_df.iloc[i]['answer']}")


Sample labels from train_df:
Example 0: Label - 0, Answer - Salesman: Great! Before I can recommend the ideal gaming console for you, could you let me know what types of games you enjoy playing and if there are any specific features you are looking for in a console?
Example 1: Label - 1, Answer - Salesman: Absolutely! We have a range of supplements that are known to support joint health and reduce inflammation. I can provide you with more information on how they work and their benefits.
Example 2: Label - 2, Answer - Salesman: Great! Im glad you reached out. My name is John, and Id be happy to assist you. How can I help?
Example 3: Label - 3, Answer - Salesman: Great question! We differentiate ourselves by placing a strong emphasis on building rapport with our clients. Our dedicated team ensures that you feel understood, valued, and supported throughout your investment journey. We also provide ongoing support and valuable resources to help you achieve your financial goals.
Example 4: 

In [9]:
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=True)


In [35]:
# clean text data for training and validation sets
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [12]:
# Convert DataFrames to Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

print("Train Dataset sample:", train_dataset[0])
print("Val Dataset sample:", val_dataset[0])
print("Test Dataset sample:", test_dataset[0])

Train Dataset sample: {'instruction': 'You are the salesman. Answer the customer queries accordingly.', 'question': 'Customer: Im interested in purchasing a gaming console. Which one would you recommend?', 'answer': 'Salesman: Great! Before I can recommend the ideal gaming console for you, could you let me know what types of games you enjoy playing and if there are any specific features you are looking for in a console?', 'label': 0, '__index_level_0__': 5}
Val Dataset sample: {'instruction': 'You are the salesman. Answer the customer queries accordingly.', 'question': 'Customer: Im looking for natural remedies for stress and anxiety. Can you suggest anything?', 'answer': 'Salesman: Managing stress and anxiety is crucial for overall well-being. We have a variety of natural remedies, such as herbal supplements and relaxation techniques, that can help you find relief. Lets explore these options together.', 'label': 349, '__index_level_0__': 321}
Test Dataset sample: {'instruction': 'You 

In [27]:
# set up the tokenization function
def tokenize_fast(examples):
    try:
        logging.debug("tokenize_fast: Starting batch tokenization...")
        tokenized_inputs = tokenizer(
            examples['instruction'],
            examples['question'],
            truncation=True,
            padding='max_length',
            max_length=512
        )
        tokenized_inputs["labels"] = examples["label"]
        logging.debug("tokenize_fast: Tokenization finished successfully.")
        return tokenized_inputs
    except Exception as e:
        logging.error(f"tokenize_fast: Error encountered: {e}")
        raise


In [13]:
# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_fast, batched=True, num_proc=4)
tokenized_val = val_dataset.map(tokenize_fast, batched=True, num_proc=4)
tokenized_test = test_dataset.map(tokenize_fast, batched=True, num_proc=4)
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

print("Tokenization complete. Sample from tokenized_train: \n", tokenized_train[0])

Map (num_proc=4):   0%|          | 0/350 [00:00<?, ? examples/s]2024-10-30 13:27:08,719 - DEBUG - tokenize_fast: Starting batch tokenization...
2024-10-30 13:27:08,736 - DEBUG - tokenize_fast: Starting batch tokenization...
2024-10-30 13:27:08,753 - DEBUG - tokenize_fast: Starting batch tokenization...
2024-10-30 13:27:08,780 - DEBUG - tokenize_fast: Starting batch tokenization...
2024-10-30 13:27:08,786 - DEBUG - tokenize_fast: Tokenization finished successfully.
2024-10-30 13:27:08,807 - DEBUG - tokenize_fast: Tokenization finished successfully.
2024-10-30 13:27:08,833 - DEBUG - tokenize_fast: Tokenization finished successfully.
2024-10-30 13:27:08,875 - DEBUG - tokenize_fast: Tokenization finished successfully.
Map (num_proc=4):   0%|          | 0/75 [00:00<?, ? examples/s]2024-10-30 13:27:09,541 - DEBUG - tokenize_fast: Starting batch tokenization...
2024-10-30 13:27:09,574 - DEBUG - tokenize_fast: Tokenization finished successfully.
2024-10-30 13:27:09,574 - DEBUG - tokenize_fast:

Tokenization complete. Sample from tokenized_train: 
 {'input_ids': tensor([  101,  2017,  2024,  1996, 18968,  1012,  3437,  1996,  8013, 10861,
         5134, 11914,  1012,   102,  8013,  1024, 10047,  4699,  1999, 13131,
         1037, 10355, 10122,  1012,  2029,  2028,  2052,  2017, 16755,  1029,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,    

In [15]:
num_labels = len(label_mapping)
model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

print(f"Number of labels in model: {model.num_labels}")
print(f"Number of unique labels in dataset: {len(set(tokenized_train['labels'].tolist()))}")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)


2024-10-30 13:28:38,752 - DEBUG - https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/11" 200 0
2024-10-30 13:28:39,024 - DEBUG - https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/11" 200 0
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of labels in model: 496
Number of unique labels in dataset: 349


/home/kira/Documents/temporary_solutions/chatbot/chatbot_env/lib64/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:

#Model
def check_labels(dataset):
    """Checks if all labels in the dataset are within the valid range (0 to num_labels - 1)."""
    labels = dataset['labels'].tolist()
    invalid_labels = [label for label in labels if label < 0 or label >= model.num_labels]
    if invalid_labels:
        print(f"Found invalid labels: {invalid_labels}")
        print("Labels must be in the range 0 to", model.num_labels - 1)
        return False
    return True

if not check_labels(tokenized_train):
    raise ValueError("Invalid labels found in training dataset")
if not check_labels(tokenized_val):
    raise ValueError("Invalid labels found in validation dataset")
if not check_labels(tokenized_test):
    raise ValueError("Invalid labels found in test dataset")

In [17]:
# build custom trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        if labels is None:
            raise ValueError("Labels should not be None")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        logits = logits.contiguous()
        labels = labels.contiguous()

        loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss



In [26]:
# set up a custom trainer
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    callbacks=[early_stopping_callback] 

)

In [18]:
trainer.train()

  0%|          | 0/264 [00:00<?, ?it/s]

                                                  
 17%|█▋        | 44/264 [25:48<1:48:25, 29.57s/it]

{'eval_loss': 6.246739387512207, 'eval_runtime': 75.8899, 'eval_samples_per_second': 0.988, 'eval_steps_per_second': 0.132, 'epoch': 1.0}


                                                  
 33%|███▎      | 88/264 [52:38<1:37:59, 33.40s/it]

{'eval_loss': 6.297903537750244, 'eval_runtime': 85.1021, 'eval_samples_per_second': 0.881, 'eval_steps_per_second': 0.118, 'epoch': 2.0}


...

                                                     
 67%|██████▋   | 176/264 [1:47:25<48:46, 33.25s/it]

{'eval_loss': 6.433151721954346, 'eval_runtime': 81.8612, 'eval_samples_per_second': 0.916, 'eval_steps_per_second': 0.122, 'epoch': 4.0}


 67%|██████▋   | 176/264 [1:47:27<53:43, 36.64s/it]

{'train_runtime': 6447.8244, 'train_samples_per_second': 0.326, 'train_steps_per_second': 0.041, 'train_loss': 6.202252474698153, 'epoch': 4.0}


TrainOutput(global_step=176, training_loss=6.202252474698153, metrics={'train_runtime': 6447.8244, 'train_samples_per_second': 0.326, 'train_steps_per_second': 0.041, 'total_flos': 369989291212800.0, 'train_loss': 6.202252474698153, 'epoch': 4.0})

In [19]:
# save the model
model_path = "./saved_model"
os.makedirs(model_path, exist_ok=True)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [38]:
# predict response
def predict_response(instruction, input_text):
    model.to(device)
    cleaned_instruction = clean_text(instruction)
    cleaned_input = clean_text(input_text)
    inputs = tokenizer(f"{cleaned_instruction} {cleaned_input}", return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    logits = outputs.logits
    predicted_id = torch.argmax(logits, dim=-1).item()
    prediction = list(label_mapping.keys())[list(label_mapping.values()).index(predicted_id)]
    semantic_results = semantic_search(f"{cleaned_instruction} {cleaned_input}")
    print(f"Semantic Search Results: {semantic_results}")
    
    if prediction in [result[0] for result in semantic_results]:
        return prediction
    else:
        return semantic_results[0][0]


In [44]:
def main():
    instruction_input = "You are a salesman chatbot, please answer the sales questions based on the provided description."

    print("Welcome to the Sales Chatbot! Type 'exit' to quit.\n")

    while True:
        sample_input = input("Please enter your question: ")
        
        if sample_input.lower() == 'exit':
            print("Goodbye!")
            break
        
        predicted_label = predict_response(instruction_input, sample_input)

        print(f"\nInstruction: {instruction_input}")
        print(f"Question: {sample_input}")
        print(f"Answer (Prediction): {predicted_label}\n")

if __name__ == "__main__":
    main()


Welcome to the Sales Chatbot! Type 'exit' to quit.



Batches: 100%|██████████| 1/1 [00:00<00:00, 31.38it/s]


Semantic Search Results: [('Salesman: Of course! Id be happy to assist you. What specific requirements do you have in mind for your new laptop?', 0.7182983), ('Salesman: Of course! Id be happy to assist you. Tell me, what are your specific requirements and preferences for a laptop?', 0.715651), ('Salesman: Of course! Id be happy to assist you. Tell me, what are your specific requirements and needs for a laptop?', 0.7136535), ('Salesman: Of course! Id be happy to assist you. What are your specific requirements and preferences for a laptop?', 0.71259975), ('Salesman: Of course! Id be happy to assist you. Tell me more about your requirements and what youre looking for in a laptop.', 0.7074313)]

Instruction: You are a salesman chatbot, please answer the sales questions based on the provided description.
Question: Hey! can you help me buy a new laptop?
Answer (Prediction): Salesman: Of course! Id be happy to assist you. What specific requirements do you have in mind for your new laptop?



Batches: 100%|██████████| 1/1 [00:00<00:00, 23.52it/s]


Semantic Search Results: [('Salesman: Of course! Id be happy to help. Can you share your preferences and requirements for a smartphone?', 0.670928), ('Salesman: Of course! I understand how frustrating that can be. Lets discuss your needs and find the perfect phone for you.', 0.65829486), ('Salesman: Of course, Id be happy to assist you. May I ask what features are most important to you in a smartphone?', 0.5965376), ('Salesman: Great! Id be happy to assist you. May I ask what features you are looking for in a smartphone?', 0.5934113), ('Salesman: I understand your frustration. It can be really frustrating when technology doesnt work as expected. Lets explore some alternative smartphone options that can better meet your needs and provide a smoother experience.', 0.53704876)]

Instruction: You are a salesman chatbot, please answer the sales questions based on the provided description.
Question: Nevermind , I want to buy a phone instead.
Answer (Prediction): Salesman: Of course! Id be hap

Batches: 100%|██████████| 1/1 [00:00<00:00, 46.06it/s]


Semantic Search Results: [('Salesman: I understand your preference for a good camera. Let me show you our latest models with advanced camera features.', 0.60999537), ('Salesman: Absolutely! Understanding your needs is important. Could you let me know what other features you prioritize besides the camera?', 0.6057595), ('Salesman: I understand the importance of having a smartphone with a superior camera. Lets explore some options that meet your requirements and discuss their features in detail.', 0.6030525), ('Salesman: Great! It would be helpful to know more about your photography needs and preferences. Could you share some details about your project?', 0.5745277), ('Salesman: Of course! Id be happy to provide you with all the information you need. But first, may I ask what specific features youre looking for in a tech product?', 0.56919056)]

Instruction: You are a salesman chatbot, please answer the sales questions based on the provided description.
Question: good cameras
Answer (Pre

Batches: 100%|██████████| 1/1 [00:00<00:00, 43.92it/s]


Semantic Search Results: [('Salesman: Of course! Id be happy to provide more information. May I ask what specifically caught your interest?', 0.63908386), ('Salesman: Youre welcome! If you have any further questions or need any additional information, feel free to reach out to me anytime. Have a great day!', 0.613834), ('Salesman: Of course! Id be happy to assist you. Tell me more about what youre looking for.', 0.6062995), ('Salesman: Of course! Id be happy to assist you. Tell me, what are your specific requirements and preferences?', 0.6023723), ('Salesman: Of course! Id be happy to assist you. Tell me more about your requirements and preferences.', 0.598987)]

Instruction: You are a salesman chatbot, please answer the sales questions based on the provided description.
Question: 
Answer (Prediction): Salesman: Of course! Id be happy to provide more information. May I ask what specifically caught your interest?

Goodbye!


note: I forgot to remove the salesman and customer from the data i used to train, but training was taking a lot of time and resources which I am struggling with currently.